In [ ]:
Install Pandas

In [ ]:
Install ccxt

In [1]:
import pandas as pd
import time
from datetime import datetime
import math
import ccxt
import json

In [2]:
exchange_id = 'binance'
exchange_class = getattr(ccxt, exchange_id)

exchange = exchange_class({
     "apiKey": "PasteYouAPI",
    "secret": "PasteSecretKey",
})

In [3]:
markets = exchange.fetchMarkets()
market_symbols = [market['symbol'] for market in markets if market['type']=='spot']

In [4]:
def create_chain(base_currency, token1, token2, buy_t1, buy_t2_by_t1, sell_t2, res):   
    return {
        'base': base_currency, # USDT                 
        'token1':token1,      # BTC                 
        'token2':token2,      # ETH       
        'buy_t1': buy_t1,     
        'buy_t2_by_t1': buy_t2_by_t1,       
        'sell_t2': sell_t2,
        'Result' : res
    }
    

def get_chains(market_symbols, quotes, base_currency='USDT'):
    chains = []

    for symbol in market_symbols:
        try:
            token1 = symbol.split('/')[0] 
            token2 = symbol.split('/')[1]
            #ищем промежуточное звено треугольника и проверяем результат в 2 стороны
            if (token1!=base_currency and token2!=base_currency):
                start = quotes[f'{token1}/{base_currency}']
                middle = quotes[f'{token1}/{token2}']
                finish = quotes[f'{token2}/{base_currency}']
                if  start['ask']!=0 and start['bid']!=0 and middle['ask']!=0 and middle['bid']!=0 and finish['ask']!=0 and finish['bid']!=0:
                    #USDT-ETH-BTC
                    chains.append(
                                create_chain(
                                    base_currency, #USDT
                                    token1,        #ETH
                                    token2,        #BTC
                                    start['ask'],  #ETH/USDT ASK
                                    middle['bid'], #ETH/BTC BID
                                    finish['bid'], #BTC/USDT BID
                                    float(100.0*middle['bid']*finish['bid']/start['ask']-100.0)
                                )
                    )
                    #USDT-BTC-ETH
                    chains.append(
                                create_chain(
                                    base_currency, #USDT
                                    token2,        #BTC
                                    token1,        #ETH
                                    finish['ask'], #BTC/USDT ASK
                                    middle['ask'], ##ETH/BTC ASK
                                    start['bid'],  #ETH/USDT BID
                                    float(100.0*start['bid']/finish['ask']/middle['ask']-100.0)
                                )
                   )

        except KeyError:
            pass
    return pd.DataFrame(chains)


In [5]:
while(1):    
    quotes = exchange.fetchBidsAsks()
    chains = get_chains(market_symbols, quotes, 'USDT')
    sorted_chains = chains.sort_values(by='Result', ascending=False)
    profit = sorted_chains.loc[sorted_chains['Result']>0.2]
    if not profit.empty:
        print(f"Time-{datetime.now().strftime('%H:%M:%S')}")
        print(profit)
    time.sleep(1)

Time-23:42:40
     base token1 token2  buy_t1  buy_t2_by_t1  sell_t2    Result
758  USDT  ALPHA    BNB  0.1342      0.000449    299.6  0.238748
Time-23:49:28
     base token1 token2  buy_t1  buy_t2_by_t1  sell_t2    Result
171  USDT    ETH    BLZ  1683.0      0.000078   0.1315  0.740438
Time-23:49:37
     base token1 token2  buy_t1  buy_t2_by_t1   sell_t2    Result
168  USDT    BLZ    BTC  0.1254      0.000006  21710.71  0.416362
Time-23:49:44
     base token1 token2  buy_t1  buy_t2_by_t1  sell_t2    Result
615  USDT   BUSD    BLZ     1.0        0.1376    0.138  0.290698
Time-23:50:51
     base token1 token2   buy_t1  buy_t2_by_t1  sell_t2    Result
169  USDT    BTC    BLZ  21723.5      0.000007   0.1577  0.545977
Time-23:51:40
     base token1 token2  buy_t1  buy_t2_by_t1  sell_t2   Result
614  USDT    BLZ   BUSD  0.1778        0.1812   0.9999  1.90207
Time-23:51:57
     base token1 token2  buy_t1  buy_t2_by_t1  sell_t2    Result
615  USDT   BUSD    BLZ     1.0        0.1842   0.1846 

KeyboardInterrupt: 

In [ ]:
#token1 - token2
100.0*0.674300*1.0008/0.67410-100.0

In [ ]:
#token2 - token1
100.0*0.513/19727.1600/0.000026-100.0